# Personal Assistant Testing

Test the email assistant with HITL and memory using deepagents.

In [ ]:
from personal_assistant import create_email_assistant
from personal_assistant.ntbk_utils import format_messages, show_prompt

## Setup

Create the email assistant agent.

In [2]:
# Create agent
agent = create_email_assistant()
print("✓ Agent created successfully")

✓ Agent created successfully


## Example 1: Meeting Request

Test the agent with a meeting request email.

In [9]:
# Helper function to resume with a decision
from langgraph.types import Command

def resume_with_decision(agent, config, decision_type="accept", edited_args=None, feedback=None):
    """Resume the agent after an interrupt with a user decision.
    
    Args:
        agent: The agent instance
        config: Config with thread_id
        decision_type: "accept", "edit", "ignore", or "response"
        edited_args: For "edit" - the modified arguments
        feedback: For "response" - user feedback text
    """
    if decision_type == "accept":
        decision = {"type": "accept"}
    elif decision_type == "edit":
        decision = {"type": "edit", "args": {"args": edited_args}}
    elif decision_type == "ignore":
        decision = {"type": "ignore"}
    elif decision_type == "response":
        decision = {"type": "response", "args": feedback}
    else:
        raise ValueError(f"Invalid decision type: {decision_type}")
    
    # Resume with the decision
    result = agent.invoke(
        Command(resume=[decision]),
        config=config,
    )
    return result
 
print("✓ Helper function loaded")

✓ Helper function loaded


In [ ]:
# Example email as markdown string
email_markdown = """
**Subject**: Quick question about next week
**From**: jane@example.com
**To**: lance@langchain.dev

Hi Lance,

Can we meet next Tuesday at 2pm to discuss the project roadmap?

Best,
Jane

---
"""

# Display email with rich formatting
show_prompt(email_markdown, title="📧 Email to Process", border_style="cyan")

In [ ]:
# Configure thread
config_1 = {"configurable": {"thread_id": "test-thread-1"}}

# Invoke agent with markdown email string
result_1 = agent.invoke(
    {"messages": [{"role": "user", "content": email_markdown}]},
    config=config_1,
)

# Display result with rich formatting
print("\n" + "="*60)
print("Agent Response")
print("="*60 + "\n")

# Check for interrupts (HITL)
if "__interrupt__" in result_1:
    print("🛑 INTERRUPT: Agent is waiting for your approval\n")
    
    for interrupt in result_1["__interrupt__"]:
        for request in interrupt.value:
            print("Action:", request["action_request"]["action"])
            print("Args:", request["action_request"]["args"])
            print("\nAllowed actions:", request["config"])
            print("\nDescription:")
            show_prompt(request["description"], title="📋 Action Details", border_style="yellow")
            print("\n" + "-"*60)
    
    print("\n💡 To continue, you need to resume the agent with a decision.")
    print("   Use: agent.invoke(None, config=config_1)")
else:
    format_messages(result_1["messages"])

### Example: Resume by accepting the action

In [12]:
result_1_continued = resume_with_decision(agent, config_1, decision_type="accept")
format_messages(result_1_continued["messages"])

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│ **Subject**: Quick question about next week                                                                     │
│ **From**: jane@example.com                                                                                      │
│ **To**: lance@langchain.dev                                                                                     │
│                                                                                                                 │
│ Hi Lance,                                                                                                       │
│                                                                                                                 │
│ Can we meet next Tuesday at 2pm to discuss the project roadmap?                                                 │
│                                                                                                                 │
│ Best,                                                                                                           │
│ Jane                                                                                                            │
│                                                                                                                 │
│ ---                                                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ I'll help you handle this meeting request. Let me check your calendar availability for next Tuesday at 2pm.     │
│                                                                                                                 │
│ 🔧 Tool Call: check_calendar_availability                                                                       │
│    Args: {                                                                                                      │
│   "day": "2025-12-09"                                                                                           │
│ }                                                                                                               │
│    ID: toolu_01HLFiUonMHpa96sjmnVdTam                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Available times on 2025-12-09: 9:00 AM, 2:00 PM, 4:00 PM                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Great! You're available at 2pm on Tuesday, December 9th. Let me schedule this meeting and send a response to    │
│ Jane.                                                                                                           │
│                                                                                                                 │
│ 🔧 Tool Call: schedule_meeting                                                                                  │
│    Args: {                                                                                                      │
│   "attendees": [                                                                                                │
│     "jane@example.com"                                                                                          │
│   ],                                                                                                            │
│   "subject": "Project Roadmap Discussion",                                                                      │
│   "duration_minutes": 30,                                                                                       │
│   "preferred_day": "2025-12-09T14:00:00",                                                                       │
│   "start_time": 14                                                                                              │
│ }                                                                                                               │
│    ID: toolu_01W4G5PFq7Et4TPsXxbKVvxA                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Tool call schedule_meeting with id toolu_01W4G5PFq7Et4TPsXxbKVvxA was cancelled - another message came in       │
│ before it could be completed.                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ I'll help you handle this meeting request. Let me check your calendar availability for next Tuesday at 2pm.     │
│                                                                                                                 │
│ 🔧 Tool Call: check_calendar_availability                                                                       │
│    Args: {                                                                                                      │
│   "day": "2025-12-09"                                                                                           │
│ }                                                                                                               │
│    ID: toolu_01We2XxG49E7jtJZhPJiVBGU                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Available times on 2025-12-09: 9:00 AM, 2:00 PM, 4:00 PM                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Perfect! You're available at 2pm on Tuesday, December 9th. Let me schedule this meeting.                        │
│                                                                                                                 │
│ 🔧 Tool Call: schedule_meeting                                                                                  │
│    Args: {                                                                                                      │
│   "attendees": [                                                                                                │
│     "jane@example.com"                                                                                          │
│   ],                                                                                                            │
│   "subject": "Project Roadmap Discussion",                                                                      │
│   "duration_minutes": 30,                                                                                       │
│   "preferred_day": "2025-12-09T14:00:00",                                                                       │
│   "start_time": 14                                                                                              │
│ }                                                                                                               │
│    ID: toolu_01DPWcDw2dYc3CgiU68Gh8ST                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Meeting 'Project Roadmap Discussion' scheduled on Tuesday, December 09, 2025 at 14 for 30 minutes with 1        │
│ attendees                                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Great! Now let me send a response email to Jane confirming the meeting.                                         │
│                                                                                                                 │
│ 🔧 Tool Call: write_email                                                                                       │
│    Args: {                                                                                                      │
│   "to": "jane@example.com",                                                                                     │
│   "subject": "Re: Quick question about next week",                                                              │
│   "content": "Hi Jane,\n\nYes, Tuesday, December 9th at 2pm works perfectly for me. I've scheduled a 30-minute  │
│ meeting to discuss the project roadmap.\n\nLooking forward to it!\n\nBest,\nLance"                              │
│ }                                                                                                               │
│    ID: toolu_01WmATRwvRPPmmZkoSDkYPxt                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### Example: Resume by editing

In [15]:
edited_args = {
     "attendees": ["jane@example.com"],
     "subject": "Project Roadmap Discussion - UPDATED",
     "duration_minutes": 45,  # Changed from 30 to 45
     "preferred_day": "2025-12-09T14:00:00",
     "start_time": 14
}

result_1_continued = resume_with_decision(agent, config_1, decision_type="edit", edited_args=edited_args)
format_messages(result_1_continued["messages"])

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│ **Subject**: Quick question about next week                                                                     │
│ **From**: jane@example.com                                                                                      │
│ **To**: lance@langchain.dev                                                                                     │
│                                                                                                                 │
│ Hi Lance,                                                                                                       │
│                                                                                                                 │
│ Can we meet next Tuesday at 2pm to discuss the project roadmap?                                                 │
│                                                                                                                 │
│ Best,                                                                                                           │
│ Jane                                                                                                            │
│                                                                                                                 │
│ ---                                                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ I'll help you handle this meeting request. Let me check your calendar availability for next Tuesday at 2pm.     │
│                                                                                                                 │
│ 🔧 Tool Call: check_calendar_availability                                                                       │
│    Args: {                                                                                                      │
│   "day": "2025-12-09"                                                                                           │
│ }                                                                                                               │
│    ID: toolu_01HLFiUonMHpa96sjmnVdTam                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Available times on 2025-12-09: 9:00 AM, 2:00 PM, 4:00 PM                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Great! You're available at 2pm on Tuesday, December 9th. Let me schedule this meeting and send a response to    │
│ Jane.                                                                                                           │
│                                                                                                                 │
│ 🔧 Tool Call: schedule_meeting                                                                                  │
│    Args: {                                                                                                      │
│   "attendees": [                                                                                                │
│     "jane@example.com"                                                                                          │
│   ],                                                                                                            │
│   "subject": "Project Roadmap Discussion",                                                                      │
│   "duration_minutes": 30,                                                                                       │
│   "preferred_day": "2025-12-09T14:00:00",                                                                       │
│   "start_time": 14                                                                                              │
│ }                                                                                                               │
│    ID: toolu_01W4G5PFq7Et4TPsXxbKVvxA                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Tool call schedule_meeting with id toolu_01W4G5PFq7Et4TPsXxbKVvxA was cancelled - another message came in       │
│ before it could be completed.                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ I'll help you handle this meeting request. Let me check your calendar availability for next Tuesday at 2pm.     │
│                                                                                                                 │
│ 🔧 Tool Call: check_calendar_availability                                                                       │
│    Args: {                                                                                                      │
│   "day": "2025-12-09"                                                                                           │
│ }                                                                                                               │
│    ID: toolu_015MCueda45cdMN5XkZyRxou                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Available times on 2025-12-09: 9:00 AM, 2:00 PM, 4:00 PM                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Perfect! You're available at 2pm on Tuesday, December 9th. Let me schedule this meeting.                        │
│                                                                                                                 │
│ 🔧 Tool Call: schedule_meeting                                                                                  │
│    Args: {                                                                                                      │
│   "attendees": [                                                                                                │
│     "jane@example.com"                                                                                          │
│   ],                                                                                                            │
│   "subject": "Project Roadmap Discussion",                                                                      │
│   "duration_minutes": 30,                                                                                       │
│   "preferred_day": "2025-12-09T14:00:00",                                                                       │
│   "start_time": 14                                                                                              │
│ }                                                                                                               │
│    ID: toolu_017JvuuDJKQjTRcNbDrRURoY                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Meeting 'Project Roadmap Discussion - UPDATED' scheduled on Tuesday, December 09, 2025 at 14 for 45 minutes     │
│ with 1 attendees                                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Now let me send a response email to Jane confirming the meeting.                                                │
│                                                                                                                 │
│ 🔧 Tool Call: write_email                                                                                       │
│    Args: {                                                                                                      │
│   "to": "jane@example.com",                                                                                     │
│   "subject": "Re: Quick question about next week",                                                              │
│   "content": "Hi Jane,\n\nYes, Tuesday, December 9th at 2pm works perfectly for me. I've scheduled a 45-minute  │
│ meeting to discuss the project roadmap.\n\nLooking forward to it!\n\nBest,\nLance"                              │
│ }                                                                                                               │
│    ID: toolu_01Ckz5kut6xXCxU1PrzU9rhS                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### Example: Resume by ignoring

In [17]:
result_1_continued = resume_with_decision(agent, config_1, decision_type="ignore")
format_messages(result_1_continued["messages"])

BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'messages.24: `tool_use` ids were found without `tool_result` blocks immediately after: toolu_015WrAQ82bzbKUbET1uhfixd. Each `tool_use` block must have a corresponding `tool_result` block in the next message.'}, 'request_id': 'req_011CViX58yA9hzBebZGRqkgP'}

### Resume Example (if interrupted)

If the agent was interrupted, you can resume with a decision:

## Example 2: Simple Question

Test with a simple question that needs a direct response.

In [ ]:
# Example email as markdown string
email_markdown_2 = """
**Subject**: LangGraph documentation link
**From**: bob@example.com
**To**: lance@langchain.dev

Hey Lance,

Could you send me the link to the LangGraph docs?

Thanks!
Bob

---
"""

# Display email with rich formatting
show_prompt(email_markdown_2, title="📧 Email to Process", border_style="cyan")

In [ ]:
config_2 = {"configurable": {"thread_id": "test-thread-2"}}

# Invoke agent with markdown email string
result_2 = agent.invoke(
    {"messages": [{"role": "user", "content": email_markdown_2}]},
    config=config_2,
)

# Display result with rich formatting
print("\n" + "="*60)
print("Agent Response")
print("="*60 + "\n")

# Check for interrupts (HITL)
if "__interrupt__" in result_2:
    print("🛑 INTERRUPT: Agent is waiting for your approval\n")
    
    for interrupt in result_2["__interrupt__"]:
        for request in interrupt.value:
            print("Action:", request["action_request"]["action"])
            print("Args:", request["action_request"]["args"])
            print("\nAllowed actions:", request["config"])
            print("\nDescription:")
            show_prompt(request["description"], title="📋 Action Details", border_style="yellow")
            print("\n" + "-"*60)
    
    print("\n💡 To continue, you need to resume the agent with a decision.")
    print("   Use: agent.invoke(None, config=config_2)")
else:
    format_messages(result_2["messages"])

## Test Memory Persistence

Test that memory persists across invocations using the same thread ID.

## Example 3: Triage Functionality

The agent now uses a triage tool to classify emails before taking action. This example shows how the agent triages a marketing email.

In [ ]:
# Marketing email as markdown string
email_markdown_marketing = """
**Subject**: Limited Time Offer - 50% Off!
**From**: marketing@newsletter.com
**To**: lance@langchain.dev

Don't miss out on our amazing sale! Click here to shop now and save big on all your favorite items. This offer expires soon!

---
"""

# Display email
show_prompt(email_markdown_marketing, title="📧 Marketing Email", border_style="cyan")

# Invoke agent
config_marketing = {"configurable": {"thread_id": "test-marketing"}}
result_marketing = agent.invoke(
    {"messages": [{"role": "user", "content": email_markdown_marketing}]},
    config=config_marketing,
)

# Show triage decision
print("\n" + "="*60)
print("Agent Decision")
print("="*60 + "\n")

messages = result_marketing.get("messages", [])
for i, msg in enumerate(messages):
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        for tool_call in msg.tool_calls:
            if tool_call.get('name') == 'triage_email':
                print("🔍 TRIAGE DECISION:")
                print(f"   Classification: {tool_call.get('args', {}).get('classification', 'N/A')}")
                print(f"   Reasoning: {tool_call.get('args', {}).get('reasoning', 'N/A')}")
                print()
    if hasattr(msg, 'name') and msg.name == 'triage_email':
        print(f"✓ Result: {msg.content}")
        print()

print("Since this was classified as 'ignore', the agent calls Done immediately without drafting a response.")

In [ ]:
# Follow-up email as markdown string
email_markdown_3 = """
**Subject**: Follow-up meeting
**From**: jane@example.com
**To**: lance@langchain.dev

Hi again,

Can we also schedule a follow-up meeting next week?

Jane

---
"""

# Use same config as example 1 (same thread_id) to test memory persistence
result_3 = agent.invoke(
    {"messages": [{"role": "user", "content": email_markdown_3}]},
    config=config_1,  # Same thread as example 1
)

# Display result with rich formatting (show last 5 messages)
print("\n" + "="*60)
print("Agent Response (with memory from previous conversation)")
print("="*60 + "\n")
format_messages(result_3["messages"][-5:])